<a href="https://colab.research.google.com/github/dayaiit/Machine-Learning/blob/main/L12_Simple_Computation_Graph_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt

class ComputationNode:
    def __init__(self, name):
        self.name = name
        self.value = None
        self.grad = None  # Will store the gradient (derivative)
        self.parents = []  # Nodes that feed into this node
        self.operation = None  # Function to compute this node's value
        self.gradient_operation = None  # Function to compute gradients for parents

    def forward(self):
        # Compute this node's value based on its operation and parents
        if self.operation:
            self.value = self.operation()
        return self.value

    def backward(self, grad=1.0):
        # This gradient represents how much the final output changes with respect to this node
        self.grad = grad

        # If this node has a gradient operation, use it to compute gradients for its parents
        if self.gradient_operation:
            self.gradient_operation(grad)

# Let's implement our simple neural network example
def create_simple_network(w=2, b=8, x=-2, y=2):
    # Create nodes
    w_node = ComputationNode("w")
    w_node.value = w

    x_node = ComputationNode("x")
    x_node.value = x

    b_node = ComputationNode("b")
    b_node.value = b

    y_node = ComputationNode("y")
    y_node.value = y

    # c = w * x
    c_node = ComputationNode("c")
    c_node.parents = [w_node, x_node]
    c_node.operation = lambda: w_node.value * x_node.value
    c_node.gradient_operation = lambda grad: (
        w_node.backward(grad * x_node.value),  # dC/dw = x
        x_node.backward(grad * w_node.value)   # dC/dx = w
    )

    # a = c + b
    a_node = ComputationNode("a")
    a_node.parents = [c_node, b_node]
    a_node.operation = lambda: c_node.value + b_node.value
    a_node.gradient_operation = lambda grad: (
        c_node.backward(grad * 1),  # dA/dC = 1
        b_node.backward(grad * 1)   # dA/dB = 1
    )

    # d = a - y
    d_node = ComputationNode("d")
    d_node.parents = [a_node, y_node]
    d_node.operation = lambda: a_node.value - y_node.value
    d_node.gradient_operation = lambda grad: (
        a_node.backward(grad * 1),   # dD/dA = 1
        y_node.backward(grad * -1)   # dD/dY = -1
    )

    # J = 0.5 * d^2
    j_node = ComputationNode("J")
    j_node.parents = [d_node]
    j_node.operation = lambda: 0.5 * d_node.value**2
    j_node.gradient_operation = lambda grad: (
        d_node.backward(grad * d_node.value)  # dJ/dD = d
    )

    # Store all nodes in a list for easy access
    nodes = [w_node, x_node, b_node, y_node, c_node, a_node, d_node, j_node]

    return nodes

# Create the network
nodes = create_simple_network()

# Forward pass - calculate the cost J
for node in nodes:
    node.forward()
    print(f"{node.name} = {node.value}")

print("\n--- Forward pass complete ---\n")

# Backward pass - calculate all gradients
nodes[-1].backward()  # Start backpropagation from the final node (J)

# Print gradients
for node in nodes:
    print(f"dJ/d{node.name} = {node.grad}")

print("\n--- Verification ---")
# Verify using the direct formula
w, x, b, y = 2, -2, 8, 2
J_original = 0.5 * ((w * x + b) - y)**2
print(f"Original J = {J_original}")

# Calculate J with a small change in w
epsilon = 0.001
J_new = 0.5 * (((w + epsilon) * x + b) - y)**2
print(f"J with w+epsilon = {J_new}")
print(f"Approximate gradient dJ/dw = {(J_new - J_original) / epsilon}")

w = 2
x = -2
b = 8
y = 2
c = -4
a = 4
d = 2
J = 2.0

--- Forward pass complete ---

dJ/dw = -4.0
dJ/dx = 4.0
dJ/db = 2.0
dJ/dy = -2.0
dJ/dc = 2.0
dJ/da = 2.0
dJ/dd = 2.0
dJ/dJ = 1.0

--- Verification ---
Original J = 2.0
J with w+epsilon = 1.9960020000000005
Approximate gradient dJ/dw = -3.997999999999502
